# Bake-off: Stanford Sentiment Treebank

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

## Contents

0. [Overview](#Overview)
0. [Bake-off submission](#Bake-off-submission)
0. [Methodological note](#Methodological-note)
0. [Set-up](#Set-up)
0. [Baseline](#Baseline)
0. [TfRNNClassifier wrapper](#TfRNNClassifier-wrapper)
0. [TreeNN wrapper](#TreeNN-wrapper)

## Overview

The goal of this in-class bake-off is to __achieve the highest average F1 score__ on the SST development set, with the binary class function.

The only restriction: __you cannot make any use of the subtree labels__.

## Bake-off submission

1. A description of the model you created.
1. The value of `f1-score` in the `avg / total` row of the classification report.

Submission URL: https://canvas.stanford.edu/courses/83399/assignments/129640

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). However, if you don't use `sst.experiment` as below, then make sure you're training only on `train`, evaluating on `dev`, and that you report with 

```
from sklearn.metrics import classification_report
classification_report(y_dev, predictions)
```
where `y_dev = [y for tree, y in sst.dev_reader(class_func=sst.binary_class_func)]`

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [35]:
from collections import Counter
from rnn_classifier import RNNClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import sst, utils
import sklearn
import sklearn.naive_bayes
import tensorflow as tf
from tf_rnn_classifier import TfRNNClassifier
from tree_nn import TreeNN
import os
import numpy as np

## Baseline

In [8]:
def unigrams_phi(tree):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    return Counter(tree.leaves())

In [9]:
def fit_maxent_classifier(X, y):        
    mod = LogisticRegression(fit_intercept=True)
    mod.fit(X, y)
    return mod

In [10]:
_ = sst.experiment(
    unigrams_phi,                      # Free to write your own!
    fit_maxent_classifier,             # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed.
    class_func=sst.binary_class_func)  # Fixed.

Accuracy: 0.772
             precision    recall  f1-score   support

   negative      0.783     0.741     0.761       428
   positive      0.762     0.802     0.782       444

avg / total      0.772     0.772     0.772       872



## TfRNNClassifier wrapper

In [11]:
def rnn_phi(tree):
    return tree.leaves()    

In [12]:
def fit_tf_rnn_classifier(X, y):
    vocab = sst.get_vocab(X, n_words=3000)
    mod = TfRNNClassifier(
        vocab, 
        eta=0.05,
        batch_size=2048,
        embed_dim=50,
        hidden_dim=50,
        max_length=52, 
        max_iter=500,
        cell_class=tf.nn.rnn_cell.LSTMCell,
        hidden_activation=tf.nn.tanh,
        train_embedding=True)
    mod.fit(X, y)
    return mod

In [ ]:
_ = sst.experiment(
    rnn_phi,
    fit_tf_rnn_classifier, 
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_reader=sst.dev_reader)

## TreeNN wrapper

In [14]:
def tree_phi(tree):
    return tree

In [15]:
def fit_tree_nn_classifier(X, y):
    vocab = sst.get_vocab(X, n_words=3000)
    mod = TreeNN(
        vocab, 
        embed_dim=100, 
        max_iter=100)
    mod.fit(X, y)
    return mod

In [16]:
_ = sst.experiment(
    rnn_phi,
    fit_tree_nn_classifier, 
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_reader=sst.dev_reader)

KeyboardInterrupt: 

## My Submission V_1

In [17]:
def run_and_test(phi,classifier,verbose=True):
    print(phi,classifier)
    print("-----------------------------")
    score = sst.experiment(
    phi,                      # Free to write your own!
    classifier,             # Free to write your own!
    train_reader=sst.train_reader,     # Fixed by the competition.
    assess_reader=sst.dev_reader,      # Fixed.
    class_func=sst.binary_class_func,   # Fixed.
    verbose=verbose)  
    print("-----------------------------")
    return score

In [42]:
def k_grams(tree,k):
    assert type(k) is int
    words = ["<S>"]+list(tree.leaves())+["</S>"]
    k_grams = [tuple(words[i:i+k]) for i in range(len(words)-k)]
    return Counter(k_grams)

k_grams(next(sst.train_reader())[0],1)

Counter({("''",): 1,
         ("'s",): 2,
         (',',): 1,
         ('.',): 1,
         ('21st',): 1,
         ('<S>',): 1,
         ('Arnold',): 1,
         ('Century',): 1,
         ('Conan',): 1,
         ('Damme',): 1,
         ('Jean-Claud',): 1,
         ('Rock',): 1,
         ('Schwarzenegger',): 1,
         ('Segal',): 1,
         ('Steven',): 1,
         ('The',): 1,
         ('Van',): 1,
         ('``',): 1,
         ('a',): 1,
         ('and',): 1,
         ('be',): 1,
         ('destined',): 1,
         ('even',): 1,
         ('going',): 1,
         ('greater',): 1,
         ('he',): 1,
         ('is',): 1,
         ('make',): 1,
         ('new',): 1,
         ('or',): 1,
         ('splash',): 1,
         ('than',): 1,
         ('that',): 1,
         ('the',): 1,
         ('to',): 2})

In [27]:
def fit_nb_classifier_with_crossvalidation(X,y):
    model = sklearn.naive_bayes.MultinomialNB()
    param_grid = {'alpha':[0.1, 0.5, 1.0, 2.0]}
    return sst.fit_classifier_with_crossvalidation(X, y, model, 3, param_grid)

In [28]:
gram_funcs = [lambda x: k_grams(x,i) for i in range(5)]
classifier_funcs = [fit_nb_classifier_with_crossvalidation]
for phi in gram_funcs:
    for clf_func in classifier_funcs:
        run_and_test(phi,clf_func)

<function <listcomp>.<lambda> at 0x12b679730> <function fit_nb_classifier_with_crossvalidation at 0x120045488>
-----------------------------
Best params {'alpha': 2.0}
Best score: 0.515
Accuracy: 0.578
             precision    recall  f1-score   support

   negative      0.665     0.283     0.397       428
   positive      0.555     0.863     0.675       444

avg / total      0.609     0.578     0.539       872

-----------------------------
<function <listcomp>.<lambda> at 0x12b6796a8> <function fit_nb_classifier_with_crossvalidation at 0x120045488>
-----------------------------
Best params {'alpha': 2.0}
Best score: 0.515
Accuracy: 0.578
             precision    recall  f1-score   support

   negative      0.665     0.283     0.397       428
   positive      0.555     0.863     0.675       444

avg / total      0.609     0.578     0.539       872

-----------------------------
<function <listcomp>.<lambda> at 0x12b679620> <function fit_nb_classifier_with_crossvalidation at 0x120045

## My Submission V_2

In [33]:
vsmdata_home = 'vsmdata'
glove_home = os.path.join(vsmdata_home, 'glove.6B')

glove_lookup = utils.glove2dict(
    os.path.join(glove_home, 'glove.6B.50d.txt'))

In [36]:
def vsm_leaves_phi(tree, lookup, np_func=np.sum):
    """Represent tree as a combination of the vector of its words.
    
    Parameters
    ----------
    tree : nltk.Tree   
    lookup : dict
        From words to vectors.
    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise, 
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that 
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a 
        fixed length, no matter what the size of the tree is.
    
    Returns
    -------
    np.array, dimension `X.shape[1]`
            
    """
    dim = len(next(iter(lookup.values())))    
    allvecs = np.array([lookup[w] for w in tree.leaves() if w in lookup])
    if len(allvecs) == 0:
        feats = np.zeros(dim)
    else:       
        feats = np_func(allvecs, axis=0)      
    return feats

In [37]:
def glove_leaves_phi(tree, np_func=np.sum):
    return vsm_leaves_phi(tree, glove_lookup, np_func=np_func)

In [ ]:
run_and_test(glove_leaves_phi,fit_tree_nn_classifier)